<a href="https://colab.research.google.com/github/Rocks-n-Code/PythonCourse/blob/master/8%20-%20SupML_RF_Lithology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Machine Learning: Random Forest Regressor

## Building and mineral model from a pulsed neutron log.

A random forest model is common method comprised of a group of decision trees. We will be using scikit learn's [`RandomForestRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) estimator.

In [ ]:
pip install lasio

In [ ]:
import pandas as pd
import lasio
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import time

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle

import plotly.graph_objects as go
from plotly.subplots import make_subplots

%matplotlib inline

pd.set_option('display.max_columns',30)
pd.set_option('display.max_rows',10)

import warnings

For demonstration puropses we are going to use one pused neutron log for training data. If you were to be doing this for a deployable model you'd want to use more training data.

In [ ]:
las = lasio.read("https://raw.githubusercontent.com/Rocks-n-Code/PythonCourse/master/data/0504511399_2355551.LAS.txt")
las_df = las.df()

#Preview our data
las_df.head()

In [ ]:
#Pull the Non-Walk Curves
curves = {}
for x in las.curves[1:]:
    #Skip Pulsed Neutron Logs
    if 'WALK' in x['mnemonic']: continue

    curves[x['mnemonic']] = x['descr']

print(list(curves.keys()))

In [ ]:
#Our Possible Outcome Columns
pn_cols = { "WANH_WALK2" : 'Anhydrite',
            "WCAR_WALK2" : 'Carbonate',
            "WCLA_WALK2" : 'Clay',
            "WCOA_WALK2" : 'Coal',
            "WEVA_WALK2" : 'Salt',
            "WPYR_WALK2" : 'Pyrite',
            "WQFM_WALK2" : 'QFM',
            "WSID_WALK2" : 'Siderite'}

#Use Only Columns Have Detections
pn_usable = list(pn_cols.keys())
for col in pn_cols.keys():
    min_s = las_df[col].describe()['min']
    prec_detect = round(las_df[las_df[col] > min_s].shape[0]/las_df.shape[0]*100,0)
    if prec_detect <= 2:
        pn_usable.remove(col)

print(', '.join([pn_cols[x] for x in pn_usable]), 'found in >2% of samples.')

In [ ]:
#Drop Null Values
subset = list(curves.keys()) + list(pn_usable)
las_df.dropna(subset=subset, inplace=True)

#Seperate Our Training & Outcome Data
X = las_df[curves.keys()]
y = las_df[pn_usable]

#Split off 10% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

#Set up our model; no randomness in bootstrapping with depth & trees counts set low for demo purposes
regr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=20)

#Train our model
regr.fit(X_train, y_train)

#Test outcome
print('Model Score:',round(regr.score(X_test, y_test),3))

## Hyperparameter Tuning

Think of this as tweaking the inputs of our algorithm to enhance performance.  One of the first things we will do before repeditavely running a model is to decrease the number of parameters by choosing those with the most influence on the outcome of our model. This allows us to run multiple iterations faster to demonstrate visually what happens as we adjust our model inputs while minimizing the impact on score.  We do risk eliminating a minor predictor feature by doing this but it's contribution to accuracy will be minor.

The ultimate goal is to minimize error.  If our model is two simple we will not capture details present in the training data that represents the real world.  If we make our model too complex, or overfit, changes in our training data introduce error.  To decrease the irreducable error once the prior two are optimized we must have a more represenative dataset of the real world. Usually this means using a larger training dataset but eleminating sampling bias is also important.

In [ ]:
#Dropping non-contributers
feats = []
labels = []
for score, mnemonic in zip(regr.feature_importances_, curves.keys()):
    #Filter out curves that did not contribute first pass
    if score == 0:
        continue

    else:
        feats.append(score)
        labels.append(mnemonic)

##Visualize Those Curves' Contribution to Model
explode = [0 if x > 0.03 else 5*(1-x)**100 for x  in feats]
plt.pie(feats,labels=labels,explode=explode)
plt.show()

So with just filtering the inputs to our model to those that contributed on the first pass how does that affect the score?  

In [ ]:
#Drop Null Values
subset = labels + pn_usable
las_df.dropna(subset=subset, inplace=True)

X = las_df[labels]
y = las_df[pn_cols.keys()]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

regr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=20)
regr.fit(X_train, y_train)
print('Model Score:',round(regr.score(X_test, y_test),3))

##Visualize Those Curves' Contribution to Model
explode = [0 if x > 0.03 else 5*(1-x)**100 for x  in regr.feature_importances_]
plt.pie(regr.feature_importances_,labels=labels,explode=explode)
plt.show()


---

## Number of Trees
Now let's look at the affect of changing `n_estimators` or the number of trees in our forest.

In [ ]:
#muting warnings from here on out
warnings.simplefilter("ignore")

scores = []
times = []
estimators = [10,50,100,200,500,1000,2000]#,5000,10000] #Can add some serious time.
for esimator in estimators:
    print('Training with',esimator)
    t1 = time.time()
    regr = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=esimator)
    regr.fit(X_train, y_train)
    scores.append(regr.score(X_test, y_test))
    times.append(time.time() - t1)

#Plot Results
fig, ax1 = plt.subplots()

ax1.set_xlabel('Estimators')
ax1.set_ylabel('Score', color='b')
ax1.plot(estimators, scores, color='b')
ax1.tick_params(axis='y', labelcolor='b')

ax2 = ax1.twinx()

ax2.set_ylabel('time (s)', color='r')
ax2.scatter(estimators, times, color='r')
ax2.tick_params(axis='y', labelcolor='r')
plt.show()


---

## Depth of Trees


Now let's look at the affect of changing `max_depth` or the maximum depth of the trees in our forest. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

In [ ]:
scores = []
times = []
depths = [2,5,10,20,30,40,None]
for depth in depths:
    print('Training with',depth)
    t1 = time.time()
    regr = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=10)
    regr.fit(X_train, y_train)
    scores.append(regr.score(X_test, y_test))
    times.append(time.time() - t1)

#Replace None for purpose of plot
depths = [x if x != None else 50 for x in depths ]

#Plot Results
fig, ax1 = plt.subplots()

ax1.set_xlabel('Depth')
ax1.set_ylabel('Score', color='g')
ax1.plot(depths, scores, color='g')
ax1.tick_params(axis='y', labelcolor='g')

ax2 = ax1.twinx()

ax2.set_ylabel('time (s)', color='r')
ax2.scatter(depths, times, color='r')
ax2.tick_params(axis='y', labelcolor='r')
plt.show()

---

## Random State


`random_state` controls two items in a random forest model: the randomness of the samples that each tree gets with boostraping & the selection of features to consider when looking to split at each node. `None`(default) uses `np.random` but if we want our model to produce the same result when runnning multiple times (ie deterministic) we can use an integer to seed a random number generator. `0` and `42` are commonly used.

In [ ]:
scores = []
times = []
randomness = [0,3,7,41,42,43,None,None,None,None,None,None]
for rand in randomness:
    print('Training with',rand)
    t1 = time.time()
    regr = RandomForestRegressor(max_depth=2, random_state=rand, n_estimators=10)
    regr.fit(X_train, y_train)
    scores.append(regr.score(X_test, y_test))
    times.append(time.time() - t1)

#Replace None for purpose of plot
randomness = [x if x != None else 100+i for i,x in enumerate(randomness)]

#Plot Results
fig, ax1 = plt.subplots()

ax1.set_xlabel('Random State')
ax1.set_ylabel('Score', color='g')
ax1.plot(randomness, scores, color='g')
ax1.tick_params(axis='y', labelcolor='g')

ax2 = ax1.twinx()

ax2.set_ylabel('time (s)', color='r')
ax2.scatter(randomness, times, color='r')
ax2.tick_params(axis='y', labelcolor='r')
plt.show()

Notice the changes in score? What happens when we do not seed the random number generator?

---

## Max Features
`max_features` or the maximum number of features considered for splitting a node. This parameter also take special case strings: `'auto'`(default),`'sqrt'`, `'log2'`, `None`(default).  Generally, by having "dumb" trees you increase the variety of trees allowing for different features to show through the dominant features.

In [ ]:
scores = []
times = []
max_features = [2,5,10,X.shape[1],'sqrt','log2',None]
for mx in max_features:
    print('Training with',mx)
    t1 = time.time()
    regr = RandomForestRegressor(max_depth=2, max_features=mx,random_state=0, n_estimators=10)
    regr.fit(X_train, y_train)
    scores.append(regr.score(X_test, y_test))
    times.append(time.time() - t1)

#Replace Strings and None for purpose of plot
max_features = [x if x != None else 28 for x in max_features]
max_features = [x if x != 'log2' else 27 for x in max_features ]
max_features = [x if x != 'sqrt' else 26 for x in max_features ]
max_features = [x if x != 'auto' else 25 for x in max_features ]


#Plot Results
fig, ax1 = plt.subplots()

ax1.set_xlabel('Max Features')
ax1.set_ylabel('Score', color='k')
ax1.plot(max_features, scores, color='k')
ax1.tick_params(axis='y', labelcolor='k')

ax2 = ax1.twinx()

ax2.set_ylabel('time (s)', color='r')
ax2.scatter(max_features, times, color='r')
ax2.tick_params(axis='y', labelcolor='r')
plt.show()

What was the optimum maximum? Remember that 25-28 are the special cases.

---
## Min Features
`min_samples_split` or the minimum number of samples required to split an internal node.

In [ ]:
scores = []
times = []
min_features = [2,3,4,5]
for mn in min_features:
    print('Training with',mn)
    t1 = time.time()
    regr = RandomForestRegressor(max_depth=2, min_samples_split=mn, random_state=0, n_estimators=10)
    regr.fit(X_train, y_train)
    scores.append(regr.score(X_test, y_test))
    times.append(time.time() - t1)

#Replace Strings for purpose of plot
min_features = [x if x != 'log2' else 27 for x in min_features ]
min_features = [x if x != 'sqrt' else 26 for x in min_features ]
min_features = [x if x != 'auto' else 25 for x in min_features ]


#Plot Results
fig, ax1 = plt.subplots()

ax1.set_xlabel('Min Features')
ax1.set_ylabel('Score', color='grey')
ax1.plot(min_features, scores, color='grey')
ax1.tick_params(axis='y', labelcolor='grey')

ax2 = ax1.twinx()

ax2.set_ylabel('time (s)', color='r')
ax2.scatter(min_features, times, color='r')
ax2.tick_params(axis='y', labelcolor='r')
plt.show()

In this case we see that changing the minimum number of features to split a node doesn't have a positive affect on our model.

---

These relationships are not always independent and you can also test them against each other to get the most out of your model.

 ### <font color='red'>NOTE</font>:
 *If you <font color='red'>are not</font> running a faster computer **skip** the cell below and **load** the data with the cell below it.*

In [ ]:
#********************************************RUN*************************************************#
#To run it yourself run this cell
depths = list(range(2,21))
trees = list(range(10,2000,100))
trees = [2,3,4,5,6,7,8,9,10,25,50,75,100,150,200,250,500,1000,1500,2000]


xa = np.array([depths]*20).T
ya = np.array([trees]*19)

Z_s = []
Z_t = []
for depth in depths:
    z_s = []
    z_t = []
    for tree in trees:
        t1 = time.time()
        regr = RandomForestRegressor(max_depth=depth, random_state=0, n_estimators=tree, n_jobs=-1)
        regr.fit(X_train, y_train)
        z_t.append(time.time() - t1)
        z_s.append(regr.score(X_test, y_test))
    Z_s.append(z_s)
    Z_t.append(z_t)

Z_t = np.array(Z_t)
Z_s = np.array(Z_s)
#********************************************RUN*************************************************#

In [ ]:
#*******************************************LOAD************************************************#

#To load the pre-completed data from the cell above run this cell
depths = list(range(2,21))
trees = [2,3,4,5,6,7,8,9,10,25,50,75,100,150,200,250,500,1000,1500,2000]

xa = np.array([depths]*20).T
ya = np.array([trees]*19)

Z_t = np.loadtxt("https://raw.githubusercontent.com/Rocks-n-Code/PythonCourse/master/data/Z_t.out")
Z_s = np.loadtxt("https://raw.githubusercontent.com/Rocks-n-Code/PythonCourse/master/data/Z_s.out")
#*******************************************LOAD************************************************#

In [ ]:
##Plot the number and depth of trees with the model score and amount of time.
#Make Figure with 2 3D Subplots
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'surface'}, {'type': 'surface'}]])

#Add Subplot 1
fig.add_trace(
              go.Surface(x=xa,
                        y=ya,
                        z=Z_s,
                        colorscale='Viridis',
                        showscale=False,
                        contours = {"z": {"show": True,
                                          "start": 0.1,
                                          "end": 0.9,
                                          "size": 0.025,
                                          'color':'black'}
                                    },
                        name='Score',
                        hovertemplate ='%{y} trees with a depth of %{x}'+
                                        ' yeilds a score of %{z: .2f}.'
                        ),
              row=1,
              col=1
              )

#Add Subplot 2
fig.add_trace(
              go.Surface(x=xa,
                        y=ya,
                        z=Z_t,
                        colorscale='RdBu',
                        showscale=False,
                        name='Compute Time',
                        contours = {"z": {"show": True,
                                          "start": 1,
                                          "end": 10,
                                          "size": 0.5}
                                    },

                        hovertemplate ='%{y} trees with a depth of %{x}'+
                                       ' yeilds a compute time of %{z: .1f}.'
                        ),

              row=1,
              col=2)

#Adjust Size and Set Title
fig.update_layout(
                  title_text='Number and Depth of Trees Effect on Score & Compute Cost',
                  height=600,
                  width=1500,
                 )

#Setup Default Initial View
camera = dict(
              up=dict(x=0, y=0, z=1),
              center=dict(x=0, y=0, z=0),
              eye=dict(x=-1.75, y=-1.75, z=0.25)
          )

#Change Axis Labels on Subplot 1
fig.update_layout(scene1 = dict(xaxis_title='Tree Depth',
                                yaxis_title='N Trees',
                                zaxis_title='Score',
                                camera=camera)
                  )

#Change Axis Labels on Subplot 2
fig.update_layout(scene2 = dict(xaxis_title='Tree Depth',
                                yaxis_title='N Trees',
                                zaxis_title='Time (ms)',
                                camera=camera),
                  )

fig.show()

Drag points on the models to rotate them.
- Notice the plateau in the score that we reach around 250 trees?  
- Notice that the influence of the depth of the trees decreases with the increaded number of trees?

---

# Give it a try

With what we learned let's retrain our model. Pick inputs from what we learned above and retrain the model.  
- Adjust and try it again.
- Better or worse score?
- Was there a decrease or increase of training time?

By setting `n_jobs` to `-1` we can speed this up by using all processors.

In [ ]:
subset = labels + pn_usable

las_df.dropna(subset=subset, inplace=True)

X = las_df[labels]
y = las_df[pn_usable]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

t1 = time.time()

##Enter your values
regr = RandomForestRegressor(max_depth=2 ,
                             n_estimators=20 ,
                             max_features='sqrt' ,
                             random_state=42 ,
                             n_jobs=-1)

regr.fit(X_train, y_train)
print('Model Score:',round(regr.score(X_test, y_test),3),'in',round(time.time() - t1,3),'sec')

So we've just about doubled our model's score showing the importance of hyperameter tuning.

---


# Saving & Loading

So you want to deploy your model but don't want to train it every time?


In [ ]:
from joblib import dump, load

#Saving Model
dump(regr, 'regr.joblib')

#Loading Model
regr2 = load('regr.joblib')

Let's take a look at how you would deploy it to make predictions. Let's load a log that the model hasn't seen before and see how it does...

In [ ]:
#Load
las2 = lasio.read("https://raw.githubusercontent.com/Rocks-n-Code/PythonCourse/master/data/0512338763_3599412.las.txt")
las_df2 = las2.df()

#Drop Null Values
subset = labels + pn_usable
las_df2.dropna(subset=subset, inplace=True)

#Run Model
X = las_df2[labels][las_df2[labels].notnull()]
y_pred = regr2.predict(X)

#Write Predictions to the DF
new_cols = [x.split('_')[0] for x in pn_usable]

for i,col in enumerate(new_cols):
    las_df2[col] = y_pred[:,i]

In [ ]:
#Plot the results
c = ['blue','brown','goldenrod','gold']
x = las_df2
ztop=x.index.min(); zbot=x.index.max()

f, ax = plt.subplots(nrows=1, ncols=4, figsize=(10, 13))
for i,col in enumerate(new_cols):
    ax[i].set_ylim(ztop,zbot)
    ax[i].plot(x[col+'_WALK2'],x.index, color=c[i], alpha=0.3)
    ax[i].plot(x[col],x.index,color='grey')
    ax[i].set_title(col)
    ax[i].invert_yaxis()
    ax[i].grid(False)
    if i !=0:
        ax[i].set_yticklabels([])
ax[0].set_ylabel('MD [ft]', fontsize=16)

- Where is your model working?
- Where is it not?
- What could be done to improve the model?
- How do you think changes in fluids affects the model?